# Login and data download

In [ ]:
!pip install aicrowd-cli
API_KEY = "" 
!aicrowd login --api-key $API_KEY

!mkdir data
!mkdir assets

!aicrowd dataset download --challenge sound-prediction -j 3 -o data

# Install packages

In [ ]:
!git clone --branch v0.9.3 https://github.com/mozilla/DeepSpeech
    
%cd /content/
!sudo apt-get install python3-venv
!sudo apt-get install python3-dev
!pip install --upgrade pip
!sudo apt-get install sox
!sudo apt-get install sox libsox-fmt-mp3
!sudo apt install git
!pip install librosa==0.7.2
!sudo apt-get install pciutils
!lspci | grep -i nvidia

!wget https://github.com/git-lfs/git-lfs/releases/download/v2.11.0/git-lfs-linux-amd64-v2.11.0.tar.gz
!tar xvf /content/git-lfs-linux-amd64-v2.11.0.tar.gz -C /content
!sudo bash /content/install.sh
%cd /content/DeepSpeech
!git-lfs pull

!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.4/ds_ctcdecoder-0.7.4-cp36-cp36m-manylinux1_x86_64.whl
!pip install /content/DeepSpeech/ds_ctcdecoder-0.7.4-cp36-cp36m-manylinux1_x86_64.whl

!pip3 install folium==0.2.1
!pip3 install --upgrade pip==20.0.2 wheel==0.34.2 setuptools==46.1.3
!pip3 install --upgrade --force-reinstall -e .

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
! echo $PATH

import os
os.environ['PATH'] += ":/usr/local/cuda-10.0/bin"
os.environ['CUDADIR'] = "/usr/local/cuda-10.0"
os.environ['LD_LIBRARY_PATH'] = "/usr/lib64-nvidia:/usr/local/cuda-10.0/lib64"

!echo $PATH
!echo $LD_LIBRARY_PATH
!source ~/.bashrc

!env | grep -i cuda

%cd /content/
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-get install freeglut3 freeglut3-dev libxi-dev libxmu-dev
!sudo apt-get install build-essential dkms
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub

!sudo apt-get update
!sudo apt-get install cuda-10-0

!sudo rm /usr/local/cuda
!sudo ln -s /usr/local/cuda-10.0 /usr/local/cuda
%ls -l /usr/local/

!pip3 uninstall tensorflow -y
!pip3 install 'tensorflow-gpu==1.15.2'

# Utiliry functions and constants

In [ ]:
DATA_FOLDER = "data"

In [ ]:
def preprocess_data(df, set_name):
    df['wav_filename'] = df['SoundID'].apply(lambda x : os.path.join("/content", "data", set_name+"/" +str(x) + ".wav"))
    df['transcript'] = df['label']
    df['wav_filesize'] = 30000

    return df

# Import

In [ ]:
import pandas as pd
import re
from ast import literal_eval
import os
import librosa

# Data preprocessing

In [ ]:
!unzip /content/data/train.zip  -d /content/data/train >/dev/null
!unzip /content/data/val.zip -d /content/data/val >/dev/null
!unzip /content/data/test.zip -d /content/data/test >/dev/null

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_FOLDER, "train.csv"))
val_df = pd.read_csv(os.path.join(DATA_FOLDER, "val.csv"))
test_df = pd.read_csv(os.path.join(DATA_FOLDER, "test.csv"))

In [ ]:
train_df = preprocess_data(train_df, "train")
val_df = preprocess_data(val_df, "val")
test_df = preprocess_data(test_df, "test")

# Train model

In [ ]:
%cd DeepSpeech  

!python DeepSpeech.py --train_files ../deepspeech_train.csv --dev_files ../deepspeech_val.csv --test_files ../deepspeech_test.csv \
 --n_hidden 2048 \
--audio_sample_rate 8000 --train_batch_size 32 --dev_batch_size 32 --test_batch_size 32 \
--automatic_mixed_precision True --epochs 7 \
--test_output_file ../assets/output.txt

%cd ..

# Prediction

In [ ]:
data = open(os.path.join("assets", "output.txt"))
output = data.read()

output = literal_eval(output)

In [ ]:
SoundID = [int(sample['wav_filename'].split("/")[-1].split(".")[0])  for sample in output]
label = [sample['res']  for sample in output]

In [ ]:
test_df['SoundID'] = SoundID
test_df['label'] = label
test_df = test_df.sort_values("SoundID")

# Make submission

In [ ]:
test_df.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [ ]:
!aicrowd notebook submit -c sound-prediction -a assets --no-verify